In [1]:
cd ../

/private/home/ccaucheteux/hasson-syntaxe-vs-semantics


In [2]:
from src.preprocess_stim import get_phone_dic

In [6]:
get_phone_dic().keys()

dict_keys(['aa_B', 'aa_E', 'aa_I', 'aa_S', 'ae_B', 'ae_E', 'ae_I', 'ah_B', 'ah_E', 'ah_I', 'ah_S', 'ao_B', 'ao_E', 'ao_I', 'aw_B', 'aw_E', 'aw_I', 'ay_B', 'ay_E', 'ay_I', 'ay_S', 'b_B', 'b_E', 'b_I', 'ch_B', 'ch_E', 'ch_I', 'd_B', 'd_E', 'd_I', 'dh_B', 'dh_E', 'dh_I', 'eh_B', 'eh_I', 'eh_S', 'er_B', 'er_E', 'er_I', 'er_S', 'ey_B', 'ey_E', 'ey_I', 'ey_S', 'f_B', 'f_E', 'f_I', 'g_B', 'g_E', 'g_I', 'hh_B', 'hh_I', 'ih_B', 'ih_E', 'ih_I', 'iy_B', 'iy_E', 'iy_I', 'iy_S', 'jh_B', 'jh_E', 'jh_I', 'k_B', 'k_E', 'k_I', 'l_B', 'l_E', 'l_I', 'm_B', 'm_E', 'm_I', 'n_B', 'n_E', 'n_I', 'ng_E', 'ng_I', 'oov_S', 'ow_B', 'ow_E', 'ow_I', 'ow_S', 'oy_E', 'oy_I', 'p_B', 'p_E', 'p_I', 'r_B', 'r_E', 'r_I', 's_B', 's_E', 's_I', 'sh_B', 'sh_E', 'sh_I', 't_B', 't_E', 't_I', 'th_B', 'th_E', 'th_I', 'uh_I', 'uw_E', 'uw_I', 'v_B', 'v_E', 'v_I', 'w_B', 'w_I', 'y_B', 'y_I', 'z_B', 'z_E', 'z_I', 'zh_E', 'zh_I', ''])

# Checks lexicality

In [41]:
import sklearn
sklearn.__version__

'0.24.1'

In [ ]:
import platform
print(platform.python_version())

In [39]:
nlp = spacy.load("en_core_sm")

OSError: [E050] Can't find model 'en_core_sm'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [35]:
9439/100*1.5/60

2.35975

In [1]:
import logging

import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer


def map_word_to_inputs(words, tokenizer):
    mapping = {}
    idx = 0
    inputs = tokenizer("", return_tensors="pt", add_special_tokens=False)
    inputs = {k: v.long() for k, v in inputs.items()}
    for i, word in enumerate(words):
        word_inpt = tokenizer(word, return_tensors="pt")
        for k, v in inputs.items():
            inputs[k] = torch.cat([inputs[k], word_inpt[k]], dim=1).long()
        ntok = word_inpt[k].size(1)
        mapping[i] = torch.arange(idx, idx + ntok + 1)
        idx += ntok
    return inputs, mapping

In [6]:
cd ..

/private/home/ccaucheteux/hasson-syntaxe-vs-semantics


In [7]:
from src import paths

In [8]:
transcript_files = list(paths.gentle_path.glob("*/transcript.txt"))
tasks = [p.parent.name for p in transcript_files]
tasks

['21styear',
 'bigbang',
 'black',
 'bronx',
 'forgot',
 'friends',
 'himym',
 'lucy',
 'merlin',
 'milkywayoriginal',
 'milkywaysynonyms',
 'milkywayvodka',
 'notthefallintact',
 'pieman',
 'piemanpni',
 'prettymouth',
 'santa',
 'seinfeld',
 'shame',
 'shapesphysical',
 'shapessocial',
 'sherlock',
 'slumlordreach',
 'tunnel',
 'upintheair',
 'vinny']

In [13]:
import spacy
spacy_tokenizer = spacy.load("en_core_web_lg")

In [12]:
from src.preprocess_stim import format_text

In [14]:
spacy_tokenizer("Test is on the nat")

Test is on the nat

In [20]:
[len(i) for i in mapping.values()]

[3, 2, 2, 2, 2]

In [21]:
model_name="gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
lexical = {}
for file, task in zip(transcript_files, tasks):
    print(task)
    transcript = open(file, "r").read()
    transcript = format_text(transcript)
    words = [w.text for w in spacy_tokenizer(transcript)]
    inputs, mapping = map_word_to_inputs(["The cat", "is", "on", "the", "mat"], tokenizer)
    _, mapping = map_word_to_inputs(transcript, tokenizer)
    
    valid = [len(i)==2 for i in mapping.values()]
    lexical[task] = (np.sum(valid), len(valid), np.sum(valid)/len(valid))

21styear
bigbang
black
bronx
forgot
friends
himym
lucy
merlin
milkywayoriginal
milkywaysynonyms
milkywayvodka
notthefallintact
pieman
piemanpni
prettymouth
santa
seinfeld
shame
shapesphysical
shapessocial
sherlock
slumlordreach
tunnel
upintheair
vinny


In [23]:
lexical.keys()

dict_keys(['21styear', 'bigbang', 'black', 'bronx', 'forgot', 'friends', 'himym', 'lucy', 'merlin', 'milkywayoriginal', 'milkywaysynonyms', 'milkywayvodka', 'notthefallintact', 'pieman', 'piemanpni', 'prettymouth', 'santa', 'seinfeld', 'shame', 'shapesphysical', 'shapessocial', 'sherlock', 'slumlordreach', 'tunnel', 'upintheair', 'vinny'])

In [32]:
import pandas as pd
df = pd.DataFrame(lexical).T
df.columns = ("lexical", "all", "share")
df["sublexical"] = df["all"] - df["lexical"]
df["share"] = 1-df["share"]
df[["sublexical", "all", "share"]]

,sublexical,all,share
21styear,0.0,45573.0,0.000000
bigbang,18.0,2636.0,0.006829
black,1.0,7738.0,0.000129
bronx,60.0,7035.0,0.008529
forgot,1.0,10909.0,0.000092
friends,6.0,2555.0,0.002348
himym,14.0,2575.0,0.005437
lucy,4.0,8493.0,0.000471
merlin,72.0,11594.0,0.006210
milkywayoriginal,91.0,5731.0,0.015879


In [ ]:
lexical

In [ ]:
def get_transformer_embeddings(words, model_name="gpt2", agg="mean"):

    # Load
    # words = events.word_raw.values
    assert agg in ["sum", "mean", "last"]

    print(f"Loading model {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    # Inputs
    print("Processed words", words[:5])
    inputs, mapping = map_word_to_inputs(words, tokenizer)